https://gis.stackexchange.com/questions/461143/convert-wrf-netcdf-to-tiff

In [216]:
import os
import re
import netCDF4 as nc
import numpy as np
import rasterio
from rasterio.transform import from_origin
import wrf
import numpy as np

In [217]:
folder = 'wrfout'
for filename in os.listdir(folder):
    # Eliminar los segundos en el formato HH:MM:SS a HH:MM
    new_filename = re.sub(r'(\d{2}:\d{2}):\d{2}', r'\1', filename)
    # Reemplazar caracteres no permitidos con guiones
    new_filename = re.sub(r'[%:]', '-', new_filename)
    if new_filename != filename:
        old_path = os.path.join(folder, filename)
        new_path = os.path.join(folder, new_filename)
        os.rename(old_path, new_path)

        print(f"Renamed: {filename} -> {new_filename}")


In [ ]:
name = '/wrfout_d03_2025-03-04_12-3A00-3A01'
root = 'wrfout'
data = nc.Dataset(root + name )
dominio = "D03"
time = "20250304_1200"

### Temperature

In [219]:
t2 =  wrf.getvar(data, "T2", timeidx=0)
t2

<xarray.DataArray 'T2' (south_north: 150, west_east: 138)> Size: 83kB
array([[289.6652 , 289.67563, 289.70648, ..., 289.80908, 289.84406,
        289.88074],
       [289.63898, 289.62683, 289.6249 , ..., 289.76434, 289.79633,
        289.82947],
       [289.6557 , 289.6494 , 289.64502, ..., 289.72968, 289.75708,
        289.78265],
       ...,
       [290.63458, 290.6297 , 290.62292, ..., 290.68414, 290.68896,
        290.70145],
       [290.63696, 290.63306, 290.62802, ..., 290.6842 , 290.69113,
        290.70084],
       [290.63913, 290.6354 , 290.62955, ..., 290.69635, 290.69812,
        290.71054]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 83kB -15.85 -15.85 ... -15.33
    XLAT     (south_north, west_east) float32 83kB 27.7 27.7 ... 28.21 28.21
    XTIME    float32 4B 360.0
    Time     datetime64[ns] 8B 2025-03-04T12:00:01
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  TEMP at 2 M
    units:        K
    stagger:      
    coordinates:  XLONG XLAT XTIME
    projection:   Mercator(stand_lon=-17.017000198364258, moad_cen_lat=28.506...

### Wind

In [220]:
u10 =  wrf.getvar(data, "U10", timeidx=0)
u10

<xarray.DataArray 'U10' (south_north: 150, west_east: 138)> Size: 83kB
array([[ 1.4736044,  1.5673598,  1.621093 , ..., -2.7357612, -2.563877 ,
        -2.385783 ],
       [ 1.6917155,  1.8946724,  1.9744508, ..., -2.7700315, -2.6109319,
        -2.4386046],
       [ 1.721707 ,  1.9213685,  2.0080593, ..., -2.8171146, -2.665371 ,
        -2.4981143],
       ...,
       [-1.2716293, -1.2342383, -1.2071518, ..., -1.6272811, -1.6681428,
        -1.6878622],
       [-1.220284 , -1.1802354, -1.1549448, ..., -1.619396 , -1.6579748,
        -1.694662 ],
       [-1.1832597, -1.1439153, -1.1092579, ..., -1.6027434, -1.6397377,
        -1.7029659]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 83kB -15.85 -15.85 ... -15.33
    XLAT     (south_north, west_east) float32 83kB 27.7 27.7 ... 28.21 28.21
    XTIME    float32 4B 360.0
    Time     datetime64[ns] 8B 2025-03-04T12:00:01
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  U at 10 M
    units:        m s-1
    stagger:      
    coordinates:  XLONG XLAT XTIME
    projection:   Mercator(stand_lon=-17.017000198364258, moad_cen_lat=28.506...

In [221]:
#https://confluence.ecmwf.int/pages/viewpage.action?pageId=133262398
#https://mst.nerc.ac.uk/wind_vect_convs.html
#! The wind dirección is in convention of meteorological wind direction, which is the direction wind is coming from.

def wind_speed_direction(u10, v10):
    theta_degree = theta_degree = (180 + 180/np.pi * np.arctan2(u10, v10)) % 360
    theta_radians = np.radians(theta_degree)
    v_mod = np.sqrt(u10**2 + v10**2)
    u_vel = v_mod * np.cos(theta_radians)
    v_vel = v_mod * np.sin(theta_radians)
    return(theta_degree, v_mod, u_vel, v_vel)

In [222]:
u10 = wrf.getvar(data, 'U10', timeidx=0)  
v10 = wrf.getvar(data, 'V10', timeidx=0)

nx = data.dimensions['west_east'].size
ny = data.dimensions['south_north'].size
dt, dx, dy = data.DT, data.DX, data.DY
cen_lat, cen_lon = data.CEN_LAT, data.CEN_LON  # center of the domain
truelat1, truelat2 = data.TRUELAT1, data.TRUELAT2  # true latitudes
pole_lat, pole_lon = data.POLE_LAT, data.POLE_LON  # coordinates of the pole    

print(nx, ny, cen_lat, cen_lon)

138 150 27.95649 -15.590043


In [223]:
theta_degree, v_mod, u_vel, v_vel = wind_speed_direction(u10, v10)

### Wind by height

In [224]:
wspd =  wrf.getvar(data, "wspd", timeidx=0)
z =  wrf.getvar(data, "z", timeidx=0) #model height - [MSL] (mass grid)
hgt =  wrf.getvar(data, "HGT", timeidx=0) #Terrain Height
z_agl = z - hgt #altura sobre el terreno
z_agl

<xarray.DataArray (bottom_top: 44, south_north: 150, west_east: 138)> Size: 4MB
array([[[   25.276716,    25.276848,    25.27797 , ...,    25.271454,
            25.274193,    25.277483],
        [   25.27291 ,    25.271513,    25.271866, ...,    25.266365,
            25.268951,    25.27192 ],
        [   25.270744,    25.273838,    25.273998, ...,    25.262   ,
            25.264288,    25.26694 ],
        ...,
        [   25.336088,    25.335932,    25.335732, ...,    25.345411,
            25.3471  ,    25.347614],
        [   25.336084,    25.33605 ,    25.335976, ...,    25.345201,
            25.346296,    25.34725 ],
        [   25.33661 ,    25.33639 ,    25.336319, ...,    25.345142,
            25.3466  ,    25.347221]],

       [[   78.30602 ,    78.306496,    78.31053 , ...,    78.29786 ,
            78.30723 ,    78.31829 ],
        [   78.2939  ,    78.28912 ,    78.29079 , ...,    78.2822  ,
            78.2917  ,    78.30136 ],
        [   78.28706 ,    78.29697 ,    78.29703 , ...,    78.26838 ,
            78.276474,    78.28583 ],
...
        [19194.04    , 19194.14    , 19194.14    , ..., 19188.027   ,
         19187.89    , 19187.822   ],
        [19193.969   , 19194.059   , 19194.053   , ..., 19187.87    ,
         19187.732   , 19187.658   ],
        [19193.893   , 19193.889   , 19193.885   , ..., 19187.75    ,
         19187.613   , 19187.486   ]],

       [[20116.656   , 20116.65    , 20116.64    , ..., 20117.494   ,
         20117.553   , 20117.611   ],
        [20116.56    , 20116.53    , 20116.52    , ..., 20117.47    ,
         20117.531   , 20117.494   ],
        [20116.465   , 20116.428   , 20116.428   , ..., 20117.367   ,
         20117.408   , 20117.371   ],
        ...,
        [20133.332   , 20133.543   , 20133.62    , ..., 20123.006   ,
         20122.81    , 20122.703   ],
        [20133.215   , 20133.408   , 20133.469   , ..., 20122.963   ,
         20122.766   , 20122.648   ],
        [20133.09    , 20133.158   , 20133.22    , ..., 20122.986   ,
         20122.787   , 20122.59    ]]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 83kB -15.85 -15.85 ... -15.33
    XLAT     (south_north, west_east) float32 83kB 27.7 27.7 ... 28.21 28.21
    XTIME    float32 4B 360.0
    Time     datetime64[ns] 8B 2025-03-04T12:00:01
Dimensions without coordinates: bottom_top, south_north, west_east

In [225]:
wspd_100m = wrf.interplevel(wspd, z_agl, 50)
wspd_100m

<xarray.DataArray 'wspd_wdir_interp' (south_north: 150, west_east: 138)> Size: 83kB
array([[8.738121 , 8.798484 , 8.805753 , ..., 5.134901 , 5.052837 ,
        4.987111 ],
       [4.9622173, 5.068179 , 5.184117 , ..., 4.1006174, 4.0409613,
        3.9380453],
       [3.89882  , 3.85057  , 3.86117  , ..., 5.2955985, 5.1658416,
        5.0568976],
       ...,
       [4.5309534, 4.6480126, 4.741476 , ..., 4.8085856, 4.621331 ,
        4.450443 ],
       [4.3186817, 4.221332 , 4.141483 , ..., 6.6585646, 6.5085897,
        6.3537483],
       [6.2088056, 6.0671425, 5.927858 , ..., 7.8038344, 7.8376107,
        7.9064736]], dtype=float32)
Coordinates:
    wspd_wdir  <U4 16B 'wspd'
    XLONG      (south_north, west_east) float32 83kB -15.85 -15.85 ... -15.33
    XLAT       (south_north, west_east) float32 83kB 27.7 27.7 ... 28.21 28.21
    XTIME      float32 4B 360.0
    Time       datetime64[ns] 8B 2025-03-04T12:00:01
    level      int32 4B 50
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:      104
    units:          m s-1
    stagger:        
    coordinates:    XLONG XLAT XTIME
    projection:     Mercator(stand_lon=-17.017000198364258, moad_cen_lat=28.5...
    missing_value:  9.969209968386869e+36
    _FillValue:     9.969209968386869e+36
    vert_units:     None

### Geopotential Height

In [226]:
ph =  wrf.getvar(data, "PH", timeidx=0)
ph

<xarray.DataArray 'PH' (bottom_top_stag: 45, south_north: 150, west_east: 138)> Size: 4MB
array([[[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        ...,
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ...,
         0.0000000e+00, 0.0000000e+00, 0.0000000e+00]],

       [[5.6741667e+00, 5.6767368e+00, 5.6987510e+00, ...,
         5.5709152e+00, 5.6246581e+00, 5.6891975e+00],
        [5.5994720e+00, 5.5720644e+00, 5.5789790e+00, ...,
         5.4710636e+00, 5.5218096e+00, 5.5800500e+00],
        [5.5570002e+00, 5.6177130e+00, 5.6208520e+00, ...,
         5.3854156e+00, 5.4303226e+00, 5.4823365e+00],
...
        [6.5962046e+03, 6.5975439e+03, 6.5977690e+03, ...,
         6.5183867e+03, 6.5166235e+03, 6.5155078e+03],
        [6.5950796e+03, 6.5962827e+03, 6.5963853e+03, ...,
         6.5175674e+03, 6.5157563e+03, 6.5145244e+03],
        [6.5939106e+03, 6.5940142e+03, 6.5941362e+03, ...,
         6.5172822e+03, 6.5154229e+03, 6.5134922e+03]],

       [[6.7136895e+03, 6.7133750e+03, 6.7130845e+03, ...,
         6.7264727e+03, 6.7273628e+03, 6.7282939e+03],
        [6.7125874e+03, 6.7120459e+03, 6.7117407e+03, ...,
         6.7264136e+03, 6.7273140e+03, 6.7271338e+03],
        [6.7114746e+03, 6.7108569e+03, 6.7106123e+03, ...,
         6.7254180e+03, 6.7261123e+03, 6.7259390e+03],
        ...,
        [6.9046045e+03, 6.9074097e+03, 6.9086699e+03, ...,
         6.7798271e+03, 6.7777847e+03, 6.7767476e+03],
        [6.9034219e+03, 6.9060298e+03, 6.9071045e+03, ...,
         6.7798276e+03, 6.7777852e+03, 6.7766533e+03],
        [6.9021782e+03, 6.9033794e+03, 6.9044980e+03, ...,
         6.7805386e+03, 6.7785049e+03, 6.7765566e+03]]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 83kB -15.85 -15.85 ... -15.33
    XLAT     (south_north, west_east) float32 83kB 27.7 27.7 ... 28.21 28.21
    XTIME    float32 4B 360.0
    Time     datetime64[ns] 8B 2025-03-04T12:00:01
Dimensions without coordinates: bottom_top_stag, south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XYZ
    description:  perturbation geopotential
    units:        m2 s-2
    stagger:      Z
    coordinates:  XLONG XLAT XTIME
    projection:   Mercator(stand_lon=-17.017000198364258, moad_cen_lat=28.506...

In [227]:
phb =  wrf.getvar(data, "PHB", timeidx=0)
phb

<xarray.DataArray 'PHB' (bottom_top_stag: 45, south_north: 150, west_east: 138)> Size: 4MB
array([[[     0.     ,      0.     ,      0.     , ...,      0.     ,
              0.     ,      0.     ],
        [     0.     ,      0.     ,      0.     , ...,      0.     ,
              0.     ,      0.     ],
        [     0.     ,      0.     ,      0.     , ...,      0.     ,
              0.     ,      0.     ],
        ...,
        [     0.     ,      0.     ,      0.     , ...,      0.     ,
              0.     ,      0.     ],
        [     0.     ,      0.     ,      0.     , ...,      0.     ,
              0.     ,      0.     ],
        [     0.     ,      0.     ,      0.     , ...,      0.     ,
              0.     ,      0.     ]],

       [[   490.25504,    490.25504,    490.25504, ...,    490.25504,
            490.25504,    490.25504],
        [   490.25504,    490.25504,    490.25504, ...,    490.25504,
            490.25504,    490.25504],
        [   490.25504,    490.25504,    490.25504, ...,    490.25504,
            490.25504,    490.25504],
...
        [186297.92   , 186297.92   , 186297.92   , ..., 186297.92   ,
         186297.92   , 186297.92   ],
        [186297.92   , 186297.92   , 186297.92   , ..., 186297.92   ,
         186297.92   , 186297.92   ],
        [186297.92   , 186297.92   , 186297.92   , ..., 186297.92   ,
         186297.92   , 186297.92   ]],

       [[195217.25   , 195217.25   , 195217.25   , ..., 195217.25   ,
         195217.25   , 195217.25   ],
        [195217.25   , 195217.25   , 195217.25   , ..., 195217.25   ,
         195217.25   , 195217.25   ],
        [195217.25   , 195217.25   , 195217.25   , ..., 195217.25   ,
         195217.25   , 195217.25   ],
        ...,
        [195217.25   , 195217.25   , 195217.25   , ..., 195217.25   ,
         195217.25   , 195217.25   ],
        [195217.25   , 195217.25   , 195217.25   , ..., 195217.25   ,
         195217.25   , 195217.25   ],
        [195217.25   , 195217.25   , 195217.25   , ..., 195217.25   ,
         195217.25   , 195217.25   ]]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 83kB -15.85 -15.85 ... -15.33
    XLAT     (south_north, west_east) float32 83kB 27.7 27.7 ... 28.21 28.21
    XTIME    float32 4B 360.0
    Time     datetime64[ns] 8B 2025-03-04T12:00:01
Dimensions without coordinates: bottom_top_stag, south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XYZ
    description:  base-state geopotential
    units:        m2 s-2
    stagger:      Z
    coordinates:  XLONG XLAT XTIME
    projection:   Mercator(stand_lon=-17.017000198364258, moad_cen_lat=28.506...

In [228]:
# La altura geopotencial es una medida que toma en cuenta la variación de la gravedad con la altura y la latitud
# Se puede considerar igual que la altura física para la troposfera
z_gph = (ph + phb) / 9.81
z_gph

<xarray.DataArray (bottom_top_stag: 45, south_north: 150, west_east: 138)> Size: 4MB
array([[[    0.      ,     0.      ,     0.      , ...,     0.      ,
             0.      ,     0.      ],
        [    0.      ,     0.      ,     0.      , ...,     0.      ,
             0.      ,     0.      ],
        [    0.      ,     0.      ,     0.      , ...,     0.      ,
             0.      ,     0.      ],
        ...,
        [    0.      ,     0.      ,     0.      , ...,     0.      ,
             0.      ,     0.      ],
        [    0.      ,     0.      ,     0.      , ...,     0.      ,
             0.      ,     0.      ],
        [    0.      ,     0.      ,     0.      , ...,     0.      ,
             0.      ,     0.      ]],

       [[   50.553432,    50.553696,    50.55594 , ...,    50.542908,
            50.548386,    50.554966],
        [   50.54582 ,    50.543026,    50.54373 , ...,    50.53273 ,
            50.537903,    50.54384 ],
        [   50.54149 ,    50.547676,    50.547997, ...,    50.524   ,
            50.528576,    50.53388 ],
...
        [19663.01    , 19663.146   , 19663.168   , ..., 19655.076   ,
         19654.896   , 19654.783   ],
        [19662.895   , 19663.018   , 19663.027   , ..., 19654.992   ,
         19654.809   , 19654.684   ],
        [19662.775   , 19662.785   , 19662.799   , ..., 19654.963   ,
         19654.773   , 19654.576   ]],

       [[20584.193   , 20584.16    , 20584.13    , ..., 20585.496   ,
         20585.586   , 20585.682   ],
        [20584.08    , 20584.025   , 20583.994   , ..., 20585.488   ,
         20585.582   , 20585.564   ],
        [20583.967   , 20583.904   , 20583.879   , ..., 20585.389   ,
         20585.459   , 20585.441   ],
        ...,
        [20603.654   , 20603.94    , 20604.068   , ..., 20590.936   ,
         20590.727   , 20590.621   ],
        [20603.533   , 20603.799   , 20603.91    , ..., 20590.936   ,
         20590.727   , 20590.611   ],
        [20603.406   , 20603.53    , 20603.643   , ..., 20591.006   ,
         20590.799   , 20590.602   ]]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 83kB -15.85 -15.85 ... -15.33
    XLAT     (south_north, west_east) float32 83kB 27.7 27.7 ... 28.21 28.21
    XTIME    float32 4B 360.0
    Time     datetime64[ns] 8B 2025-03-04T12:00:01
Dimensions without coordinates: bottom_top_stag, south_north, west_east

### Height

In [229]:
# Model Height for Mass Grid (MSL, Meal Sea Level)
# Es la altura física real desde un punto de referencia (generalmente el nivel del mar). No corrige por la variación de la gravedad
hgt =  wrf.getvar(data, "HGT", timeidx=0)
hgt

<xarray.DataArray 'HGT' (south_north: 150, west_east: 138)> Size: 83kB
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 83kB -15.85 -15.85 ... -15.33
    XLAT     (south_north, west_east) float32 83kB 27.7 27.7 ... 28.21 28.21
    XTIME    float32 4B 360.0
    Time     datetime64[ns] 8B 2025-03-04T12:00:01
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  Terrain Height
    units:        m
    stagger:      
    coordinates:  XLONG XLAT XTIME
    projection:   Mercator(stand_lon=-17.017000198364258, moad_cen_lat=28.506...

#### Aspect (orientación) and slope (pendiente)

In [230]:
grad_y, grad_x = np.gradient(hgt, dy, dx)

slope = np.arctan(np.sqrt(grad_x**2 + grad_y**2))
slope_deg = np.degrees(slope)

# Aspecto (orientación del terreno)
aspect = np.arctan2(-grad_y, grad_x)
aspect_deg = np.degrees(aspect)
aspect_deg = (aspect_deg + 360) % 360

### Pressure

In [231]:
print(np.nanmin(z), np.nanmax(z))

25.126015 20135.193


In [232]:
p =  wrf.getvar(data, "pressure", timeidx=0) #presión total (perturbada + base) (hPa)

p_10m = p[0,:,:]
p_10m

<xarray.DataArray 'pressure' (south_north: 150, west_east: 138)> Size: 83kB
array([[1010.7194 , 1010.7269 , 1010.6818 , ..., 1010.2614 , 1010.25977,
        1010.258  ],
       [1010.7107 , 1010.71497, 1010.71497, ..., 1010.2652 , 1010.2529 ,
        1010.25726],
       [1010.7793 , 1010.70703, 1010.67554, ..., 1010.2738 , 1010.2659 ,
        1010.262  ],
       ...,
       [1011.0299 , 1011.02594, 1011.03186, ..., 1010.1027 , 1010.13257,
        1010.1417 ],
       [1011.0395 , 1011.031  , 1011.0302 , ..., 1010.09546, 1010.13055,
        1010.167  ],
       [1011.0409 , 1011.0405 , 1011.03864, ..., 1010.17   , 1010.1658 ,
        1010.1815 ]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 83kB -15.85 -15.85 ... -15.33
    XLAT     (south_north, west_east) float32 83kB 27.7 27.7 ... 28.21 28.21
    XTIME    float32 4B 360.0
    Time     datetime64[ns] 8B 2025-03-04T12:00:01
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:      104
    MemoryOrder:    XYZ
    description:    pressure
    units:          hPa
    stagger:        
    coordinates:    XLONG XLAT XTIME
    projection:     Mercator(stand_lon=-17.017000198364258, moad_cen_lat=28.5...
    _FillValue:     1e+20
    missing_value:  1e+20

### LAI

In [252]:
_lai =  wrf.getvar(data, "LAI", timeidx=0) 

_lai

<xarray.DataArray 'LAI' (south_north: 150, west_east: 138)> Size: 83kB
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 83kB -15.85 -15.85 ... -15.33
    XLAT     (south_north, west_east) float32 83kB 27.7 27.7 ... 28.21 28.21
    XTIME    float32 4B 360.0
    Time     datetime64[ns] 8B 2025-03-04T12:00:01
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  LEAF AREA INDEX
    units:        m-2/m-2
    stagger:      
    coordinates:  XLONG XLAT XTIME
    projection:   Mercator(stand_lon=-17.017000198364258, moad_cen_lat=28.506...

## Built Dataset

In [233]:
xmin = float(np.min(t2.XLONG).values)
xmax = float(np.max(t2.XLONG).values)
ymin = float(np.min(t2.XLAT).values)
ymax = float(np.max(t2.XLAT).values)


print( xmin, xmax, ymin, ymax)

-15.851823806762695 -15.328262329101562 27.7047119140625 28.207672119140625


In [253]:
#Extracting coordinates
latitudes = data.variables["XLAT"][0,:,:]
longitudes = data.variables["XLONG"][0,:,:]

# take a single time step
time0_T2 = data.variables["T2"][0,:,:]

# # take a signgle wind step
# time0_U10 = data.variables["U10"][0,:,:]
# time0_V10 = data.variables["V10"][0,:,:]

# defining lists for the output dataset
lat, lon, temp, wind_speed_10m, wind_direction_10m, height, slope, aspect, pressure, lai, nc = [], [], [], [], [], [], [], [], [], [], []

In [254]:
# counter variable for cell indexing
count = 1
for i in range(latitudes.shape[0]):
    for j in range(latitudes.shape[1]):        
        latitude = latitudes[i, j]
        longitude = longitudes[i, j]
        if ymin <= latitude <= ymax and xmin <= longitude <= xmax:               
            lat.append(latitude)
            lon.append(longitude)
            # convert temperature from Kelvin to Celsius
            temp.append(time0_T2[i, j])
            wind_speed_10m.append(v_mod[i, j])
            wind_direction_10m.append(theta_degree[i, j])
            height.append(hgt[i, j])
            slope.append(slope_deg[i, j])
            aspect.append(aspect_deg[i, j]) 
            pressure.append(p_10m[i, j])
            lai.append(_lai[i,j])
            nc.append(count)
            count = count + 1

In [255]:
# converting to numpy column array
A = np.transpose(np.array([nc,lon, lat, temp,wind_speed_10m, wind_direction_10m, height, slope, aspect, pressure, lai]))
A

array([[ 1.00000000e+00, -1.58518238e+01,  2.77047119e+01, ...,
         0.00000000e+00,  1.01071942e+03,  0.00000000e+00],
       [ 2.00000000e+00, -1.58480024e+01,  2.77047119e+01, ...,
         0.00000000e+00,  1.01072693e+03,  0.00000000e+00],
       [ 3.00000000e+00, -1.58441811e+01,  2.77047119e+01, ...,
         0.00000000e+00,  1.01068182e+03,  0.00000000e+00],
       ...,
       [ 2.06980000e+04, -1.53359051e+01,  2.82076721e+01, ...,
         0.00000000e+00,  1.01016998e+03,  0.00000000e+00],
       [ 2.06990000e+04, -1.53320837e+01,  2.82076721e+01, ...,
         0.00000000e+00,  1.01016577e+03,  0.00000000e+00],
       [ 2.07000000e+04, -1.53282623e+01,  2.82076721e+01, ...,
         0.00000000e+00,  1.01018152e+03,  0.00000000e+00]])

In [256]:
# writing data to csv
np.savetxt('time0_T2.csv',A,delimiter=",",header="cell_number, LON, LAT, T2_degC,WIND_SPEED_10m, WIND_DIRECTION_10m, HEIGHT, SLOPE, ASPECT, PRESSURE, LAI")

In [257]:
# # x and y resolutions
deltax = (np.max(A[:,1]) - np.min(A[:,1]))/ (nx - 1)
deltay = (np.max(A[:,2]) - np.min(A[:,2]))/ (ny - 1)

print(deltax, deltay)

0.003821616625263743 0.0033755718461619126


In [258]:
# # up left corner
minx = np.min(A[:,1])
maxy = np.max(A[:,2])

print(minx, maxy)

-15.851823806762695 28.207672119140625


In [259]:
# set the transformation for rasterio. Note that the shift operated is in order to
# make the lat,lon points to be the center of each grid cell
transform = from_origin(minx-0.5*deltax,maxy+0.5*deltay, deltax, deltay)
transform

Affine(0.003821616625263743, 0.0, -15.853734615075327,
       0.0, -0.0033755718461619126, 28.209359905063707)

## Convert to GEOTIFF

In [241]:
t2 = A[:,3]
data_band = np.flipud(t2.reshape(ny, nx))
# write it to tiff format
with rasterio.open(f'output/Temperature_{dominio}_{time}.tiff', 'w', driver='GTiff', width=nx, height=ny, count=1, dtype=t2.dtype, crs='EPSG:4326', transform=transform) as dst:
    dst.write(data_band.reshape(1, ny, nx))

In [ ]:
wind_speed = A[:,4]
data_band = np.flipud(wind_speed.reshape(ny, nx))
# write it to tiff format
with rasterio.open(f'output/Wind_Speed_{dominio}_{time}.tiff', 'w', driver='GTiff', width=nx, height=ny, count=1, dtype=wind_speed.dtype, crs='EPSG:4326', transform=transform) as dst:
    dst.write(data_band.reshape(1, ny, nx))

In [243]:
wind_direction = A[:,5]
data_band = np.flipud(wind_direction.reshape(ny, nx))
# write it to tiff format
with rasterio.open(f'output/Wind_Direction_{dominio}_{time}.tiff', 'w', driver='GTiff', width=nx, height=ny, count=1, dtype=wind_direction.dtype, crs='EPSG:4326', transform=transform) as dst:
    dst.write(data_band.reshape(1, ny, nx))

In [244]:
height = A[:,6]
data_band = np.flipud(height.reshape(ny, nx))
# write it to tiff format
with rasterio.open(f'output/Height_{dominio}_{time}.tiff', 'w', driver='GTiff', width=nx, height=ny, count=1, dtype=height.dtype, crs='EPSG:4326', transform=transform) as dst:
    dst.write(data_band.reshape(1, ny, nx))

In [245]:
slope = A[:,7]
data_band = np.flipud(slope.reshape(ny, nx))
# write it to tiff format
with rasterio.open(f'output/Slope_{dominio}_{time}.tiff', 'w', driver='GTiff', width=nx, height=ny, count=1, dtype=slope.dtype, crs='EPSG:4326', transform=transform) as dst:
    dst.write(data_band.reshape(1, ny, nx))

In [246]:
Aspect = A[:,8]
data_band = np.flipud(Aspect.reshape(ny, nx))
# write it to tiff format
with rasterio.open(f'output/Aspect_{dominio}_{time}.tiff', 'w', driver='GTiff', width=nx, height=ny, count=1, dtype=Aspect.dtype, crs='EPSG:4326', transform=transform) as dst:
    dst.write(data_band.reshape(1, ny, nx))

In [247]:
pressure = A[:,9]
data_band = np.flipud(pressure.reshape(ny, nx))
# write it to tiff format
with rasterio.open(f'output/Pressure_{dominio}_{time}.tiff', 'w', driver='GTiff', width=nx, height=ny, count=1, dtype=pressure.dtype, crs='EPSG:4326', transform=transform) as dst:
    dst.write(data_band.reshape(1, ny, nx))

In [261]:
lai = A[:,10]
data_band = np.flipud(lai.reshape(ny, nx))
# write it to tiff format
with rasterio.open(f'output/LAI{dominio}_{time}.tiff', 'w', driver='GTiff', width=nx, height=ny, count=1, dtype=lai.dtype, crs='EPSG:4326', transform=transform) as dst:
    dst.write(data_band.reshape(1, ny, nx))